## Quadratization of the system

---

[[Package page]](https://github.com/AndreyBychkov/QBee/)

QBee is a Python library for transforming systems of differential equations into a systems with quadratic right-rand side. 

Here is a online program version of the packege: [link](https://huggingface.co/spaces/Armaliltril/qbee)

In [11]:
import sympy
from qbee import *

sympy.init_printing()  # If you work in Jupyter notebook 

### Example System

---


In [13]:
x, y = functions("x, y")

system = [
    (x, sp.sin(x**2) + y),
    (y, sp.cos(x**2) + x)
]

# Currently, there is no rule to estimate an upper bound for a polynomialization in general,
# so you may have to pick it yourself, if the default value is too small;
res = polynomialize(system, upper_bound=5)
res.print()

x' = w_{1} + y
y' = w_{0} + x
w_{0}' = -2*w_{1}*x*(w_{1} + y)
w_{1}' = 2*w_{0}*x*(w_{1} + y)


### System 1

---

<div style="background-color:lightblue;margin:20px;padding:10px;border-radius:10px;border-width:2px;border-color:black;box-shadow:5px 5px;width:96%">

We have the first system that:
$$
x'=-x+ax^{3}
$$

</div>

In [5]:
a = parameters("a")
x = functions("x")

eq = - x + a * x ** 3
system = [(x, eq)]

quadr_system = polynomialize_and_quadratize(system, 
                                            new_vars_name = "w")

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Elapsed time: 0.002s.


Nodes processed: 0 nodes [00:00, ? nodes/s]


Elapsed time: 0.015s.
Quadratization result
Number of introduced variables: 1
Nodes traversed: 3
Introduced variables:
w{0} = x**2



### Expression of the system

From the result we have the quadratization:
$$
y' = -2y+2ay^{2}
$$
where $y=x^2$, then we have the following system:
$$
\begin{cases}
y' = -2y+2ay^{2}\\
x' = -x + axy
\end{cases}
$$

### System 2

---

<div style="background-color:lightblue;margin:20px;padding:10px;border-radius:10px;border-width:2px;border-color:black;box-shadow:5px 5px;width:96%">

We have the second system of following form:
$$
x'=-x^{3}
$$

</div>

In [6]:
x = functions("x")

eq = - x ** 3
system = [(x, eq)]

quadr_system = polynomialize_and_quadratize(system, 
                                            new_vars_name = "w")

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Nodes processed: 0 nodes [00:00, ? nodes/s]



Elapsed time: 0.002s.


Nodes processed: 0 nodes [00:00, ? nodes/s]


Elapsed time: 0.014s.
Quadratization result
Number of introduced variables: 1
Nodes traversed: 3
Introduced variables:
w{0} = x**2



### Expression of the system

Similarly, we have the following quadratization, where $y=x^2$:
$$
y' = -2x x' = -2x^4 = -2y^2
$$
Then we have the following system:
$$
\begin{cases}
y' = -2y^2\\
x' = -xy
\end{cases}
$$

### System 3

---

<div style="background-color:lightblue;margin:20px;padding:10px;border-radius:10px;border-width:2px;border-color:black;box-shadow:5px 5px;width:96%">

We have the third system ([Duffing equation](https://en.wikipedia.org/wiki/Duffing_equation)) of following form:
$$
x'' = kx+ax^{3}+bx'
$$
(we set $x_{0}=x$ and $x_{1}=x'$). Then we have the system of the ODEs:
$$
\begin{cases}
x_{0}'=x_1\\
x_{1}'=kx_{0}+ax_{0}^{3} + bx_1
\end{cases}
$$

</div>

In [7]:
k, a, b = parameters("k, a, b")
x0, x1 = functions("x0, x1")

eq1 = -k * x0 - a * x0**3 - b * x1
system = [
    (x1, eq1),
    (x0, x1),
]

quadr_system = polynomialize_and_quadratize(system)

Current spatial time derivatives equations:
...

Variables introduced in polynomialization:


Nodes processed: 0 nodes [00:00, ? nodes/s]



Elapsed time: 0.002s.


Nodes processed: 0 nodes [00:00, ? nodes/s]


Elapsed time: 0.014s.
Quadratization result
Number of introduced variables: 1
Nodes traversed: 3
Introduced variables:
w_{0} = x0**2



### Expression of the system

Here we only introduce one new variable $w_{0}=x_{0}^{2}$. Then we got the system of the ODEs:
$$
\begin{cases}
x_{0}'=x_1\\
x_{1}'=kx_{0}+ax_{0}w_{0} + bx_{1}\\
w_{0}' = 2x_{0}x_{0}' = 2x_{0}x_{1}
\end{cases}
$$
